<a href="https://colab.research.google.com/github/DenisKai7/invoice_generator/blob/main/malam_puncak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install library yang diperlukan
!pip install gspread pandas fpdf2 pyqrcode pypng requests pillow
!pip install --upgrade google-auth-oauthlib google-auth-httplib2

# Import library yang dibutuhkan
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
from fpdf import FPDF
import os
from google.colab import drive
from datetime import datetime
import requests
from io import BytesIO
from PIL import Image
import urllib.parse
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Autentikasi Google Colab dengan Google Sheets & Drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Inisialisasi Google Drive API
drive_service = build('drive', 'v3', credentials=creds)

# Mount Google Drive untuk menyimpan invoice
drive.mount('/content/drive')

# Buka spreadsheet
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1Eig9wEouZRPDxJvmdvNBJ8g2XYAaXGFPS-bwDtVXcK8/edit?gid=0#gid=0"
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.get_worksheet(0)

# Ambil semua data dan periksa kolom yang tersedia
expected_headers = ['KODE_INVOICE', 'Tanggal', 'email', 'KATEGORI', 'NAMA', 'NO_WA', 'KODE REFERAL', 'Quantity', 'Barcode', 'template_pesan']
records = worksheet.get_all_records(expected_headers=expected_headers)
df = pd.DataFrame.from_records(records)

# Periksa nama kolom yang sebenarnya
print("Kolom yang tersedia di spreadsheet:")
print(df.columns.tolist())

# Mapping kolom yang sesuai dengan spreadsheet
COLUMN_MAPPING = {
    'invoice_id': 'KODE_INVOICE',
    'date': 'Tanggal',
    'email': 'email',
    'category': 'KATEGORI',
    'name': 'NAMA',
    'whatsapp': 'NO_WA',
    'referral_code': 'KODE REFERAL',
    'quantity': 'Quantity',
    'barcode': 'Barcode',
    'message_template': 'template_pesan'
}

# Path logo di Google Drive (sesuaikan dengan path Anda)
LOGO_PATH = '/content/drive/MyDrive/logo/diesnatalis 1-01.png'  # Logo utama
SECONDARY_LOGO_PATH = '/content/drive/MyDrive/logo/Unipma.png'  # Logo sekunder

class InvoicePDF(FPDF):
    def __init__(self, participant_data):
        super().__init__(format='A4', orientation='P')  # A4 Portrait
        self.participant_data = participant_data
        self.add_page()

    def header(self):
        # Tambahkan background dengan efek fade
        self._add_faded_background()

        # Logo di pojok kiri atas
        try:
            if os.path.exists(LOGO_PATH):
                self.image(LOGO_PATH, x=10, y=10, w=40)
            if os.path.exists(SECONDARY_LOGO_PATH):
                self.image(SECONDARY_LOGO_PATH, x=40, y=10, w=30)
        except Exception as e:
            print(f"⚠️ Error loading logos: {e}")

        # Informasi perusahaan di kanan atas
        self.set_xy(120, 10)
        self.set_font('Arial', 'B', 16)
        self.cell(0, 8, 'Groove Specta', 0, 1, 'R')

        self.set_xy(120, 18)
        self.set_font('Arial', 'I', 10)
        self.cell(0, 6, 'Official Invoice', 0, 1, 'R')

        self.set_xy(120, 26)
        self.set_font('Arial', '', 10)
        self.multi_cell(0, 6, 'Jl. Letkol Suwarno No. 15, Kanigoro, Kec. Kartoharjo, Kota Madiun, Jawa Timur 63118', 0, 'R')

    def _add_faded_background(self):
        BG_IMAGE_PATH = '/content/drive/MyDrive/logo/Groove Specta.jpg'
        if os.path.exists(BG_IMAGE_PATH):
            try:
                self.image(BG_IMAGE_PATH, x=0, y=0, w=self.w, h=self.h)
                self.set_fill_color(255, 255, 255, 200)
                self.rect(0, 0, self.w, self.h, style='F')
            except Exception as e:
                print(f"⚠️ Error adding background: {e}")

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, 'System By : Jofanza Denis Aldida & Hafiz Prayoga', 0, 0, 'C')

    def create_invoice(self):
        # Informasi invoice
        self.set_y(70)
        self.set_font('Arial', 'B', 14)
        self.cell(0, 8, f'INVOICE #{self.participant_data[COLUMN_MAPPING["invoice_id"]]}', 0, 1, 'L')

        # Garis pemisah
        self.line(10, self.get_y(), 200, self.get_y())
        self.ln(5)

        # Informasi pelanggan
        self.set_font('Arial', 'B', 10)
        self.cell(0, 6, 'BILL TO:', 0, 1, 'L')
        self.set_font('Arial', '', 10)
        self.cell(0, 6, str(self.participant_data[COLUMN_MAPPING["name"]] or '-', 0, 1, 'L'))

        # Informasi tambahan
        self.ln(5)
        self.set_font('Arial', 'B', 10)
        self.cell(40, 6, 'Invoice Date:', 0, 0, 'L')
        self.set_font('Arial', '', 10)
        invoice_date = self.participant_data.get(COLUMN_MAPPING['date'], datetime.now().strftime("%d/%m/%Y"))
        self.cell(0, 6, str(invoice_date), 0, 1, 'L')

        self.set_font('Arial', 'B', 10)
        self.cell(40, 6, 'WhatsApp:', 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(0, 6, str(self.participant_data.get(COLUMN_MAPPING['whatsapp'], '-')), 0, 1, 'L')

        self.set_font('Arial', 'B', 10)
        self.cell(40, 6, 'Referral Code:', 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(0, 6, str(self.participant_data.get(COLUMN_MAPPING['referral_code'], '-')), 0, 1, 'L')

        # Tabel item
        self.ln(10)
        self.set_font('Arial', 'B', 10)
        self.cell(120, 8, 'Description', 1, 0, 'L')
        self.cell(20, 8, 'Qty', 1, 0, 'C')
        self.cell(30, 8, 'Price', 1, 0, 'R')
        self.cell(30, 8, 'Amount', 1, 1, 'R')

        # Data item
        self.set_font('Arial', '', 10)
        category = str(self.participant_data.get(COLUMN_MAPPING['category'], '-'))
        quantity = str(self.participant_data.get(COLUMN_MAPPING['quantity'], '1'))
        price = "Rp265,000"  # Harga default, bisa disesuaikan
        amount = "Rp265,000"  # Total default

        self.cell(120, 8, category, 1, 0, 'L')
        self.cell(20, 8, quantity, 1, 0, 'C')
        self.cell(30, 8, price, 1, 0, 'R')
        self.cell(30, 8, amount, 1, 1, 'R')

        # Total
        self.set_font('Arial', 'B', 10)
        self.cell(140, 8, 'Subtotal', 1, 0, 'R')
        self.cell(30, 8, amount, 1, 1, 'R')

        self.cell(140, 8, 'Total', 1, 0, 'R')
        self.cell(30, 8, amount, 1, 1, 'R')

        # Paid on date (gunakan tanggal dari spreadsheet jika ada)
        paid_date = self.participant_data.get(COLUMN_MAPPING['date'], datetime.now().strftime("%d %b %Y"))
        self.cell(140, 8, f'Paid on {paid_date}', 1, 0, 'R')
        self.cell(30, 8, amount, 1, 1, 'R')

        # Catatan
        self.ln(10)
        self.set_font('Arial', 'I', 8)
        self.multi_cell(0, 6, 'E-invoice ini sebagai bukti pembayaran saat penukaran ticket.\nHarap tunjukkan invoice ini beserta identitas yang valid.', 0, 'L')

        # Barcode jika ada
        if COLUMN_MAPPING["barcode"] in self.participant_data and self.participant_data[COLUMN_MAPPING["barcode"]]:
            try:
                response = requests.get(self.participant_data[COLUMN_MAPPING["barcode"]])
                img = Image.open(BytesIO(response.content))
                temp_img_path = f"/tmp/barcode_{self.participant_data[COLUMN_MAPPING['invoice_id']]}.png"
                img.save(temp_img_path)
                self.image(temp_img_path, x=150, y=self.get_y(), w=50)
                os.remove(temp_img_path)
            except Exception as e:
                print(f"⚠️ Error adding barcode: {e}")

def generate_invoice_pdf(participant_data):
    # Buat PDF
    pdf = InvoicePDF(participant_data)
    pdf.create_invoice()

    # Buat folder penyimpanan di Colab
    local_invoice_dir = "/content/invoices_groove_specta"
    os.makedirs(local_invoice_dir, exist_ok=True)

    # Simpan PDF lokal
    pdf_filename = f"INV_{participant_data[COLUMN_MAPPING['invoice_id']]}.pdf"
    local_pdf_path = f"{local_invoice_dir}/{pdf_filename}"
    pdf.output(local_pdf_path)

    return local_pdf_path, pdf_filename

def upload_to_drive(file_path, file_name, folder_id=None):
    """Upload file ke Google Drive dan set permissions"""
    file_metadata = {
        'name': file_name,
        'parents': [folder_id] if folder_id else None
    }

    media = MediaFileUpload(file_path, mimetype='application/pdf')

    file = drive_service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id,webViewLink'
    ).execute()

    # Set permissions agar bisa diakses oleh siapa saja dengan link
    permission = {
        'type': 'anyone',
        'role': 'reader'
    }

    drive_service.permissions().create(
        fileId=file['id'],
        body=permission
    ).execute()

    return file['webViewLink']

def generate_whatsapp_link(phone_number):
    if pd.isna(phone_number) or not phone_number:
        return ""

    # Format nomor WhatsApp (62XXXXXXXXXX)
    clean_number = ''.join(filter(str.isdigit, str(phone_number)))
    if clean_number.startswith('0'):
        clean_number = '62' + clean_number[1:]
    elif not clean_number.startswith('62'):
        clean_number = '62' + clean_number
    return f"https://wa.me/{clean_number}"

def generate_message_template(participant_data, drive_link):
    # Format WhatsApp number dengan +62
    whatsapp_num = str(participant_data.get(COLUMN_MAPPING['whatsapp'], ''))
    if whatsapp_num:
        clean_number = ''.join(filter(str.isdigit, whatsapp_num))
        if clean_number.startswith('0'):
            clean_number = '+62' + clean_number[1:]
        elif not clean_number.startswith('62'):
            clean_number = '+62' + clean_number
        else:
            clean_number = '+' + clean_number
    else:
        clean_number = ''

    return f"""
Halo {participant_data[COLUMN_MAPPING['name']]},

Terima kasih telah membeli tiket *Groove Specta*!
Berikut invoice resmi Anda:

🔹 *Kode Invoice:* {participant_data[COLUMN_MAPPING['invoice_id']]}
🔹 *Kategori:* {participant_data[COLUMN_MAPPING['category']}
🔹 *Jumlah Tiket:* {participant_data.get(COLUMN_MAPPING['quantity'], '1')}

📥 *Download Invoice:*
{drive_link}

*Informasi Penting:*
- Harap simpan invoice ini sebagai bukti pembayaran
- Tunjukkan invoice beserta identitas yang valid saat masuk venue
- Untuk pertanyaan, hubungi kami via WhatsApp di {clean_number}

Salam,
*Panitia Groove Specta*
"""

def process_all_participants():
    updates = []
    errors = []

    # ID folder tujuan di Google Drive (ganti dengan folder ID Anda)
    DRIVE_FOLDER_ID = '1TgkYlvaWh_Wy5heW3zaP6b6WDMbGsxKG'  # Ganti dengan folder ID tujuan

    for index, row in df.iterrows():
        try:
            invoice_id = row[COLUMN_MAPPING['invoice_id']]
            print(f"\nMemproses peserta: {row.get(COLUMN_MAPPING['name'], '')} (Invoice: {invoice_id})")

            # 1. Generate PDF
            local_pdf_path, pdf_filename = generate_invoice_pdf(row)

            # 2. Upload ke Google Drive dan dapatkan link
            drive_link = upload_to_drive(local_pdf_path, pdf_filename, DRIVE_FOLDER_ID)
            print(f"✅ Invoice diupload ke: {drive_link}")

            # 3. Generate link WhatsApp
            whatsapp_link = generate_whatsapp_link(row.get(COLUMN_MAPPING['whatsapp'], ''))
            updates.append((index, 'link_whatsapp', whatsapp_link))

            # 4. Buat template pesan
            message = generate_message_template(row, drive_link)
            updates.append((index, 'template_pesan', message))

            # 5. Bersihkan file lokal
            os.remove(local_pdf_path)

        except Exception as e:
            error_msg = f"❌ Error saat memproses {row.get(COLUMN_MAPPING['name'], '')}: {str(e)}"
            print(error_msg)
            errors.append((index, error_msg))

    # Update spreadsheet
    for update in updates:
        row_idx, col_name, value = update
        try:
            # Cari kolom yang sesuai
            col_names = [col.lower().replace(' ', '_') for col in df.columns]
            target_col = col_name.lower()

            if target_col in col_names:
                col_idx = col_names.index(target_col) + 1  # +1 karena indeks spreadsheet mulai dari 1
                worksheet.update_cell(row_idx + 2, col_idx, value)
                print(f"✔️ Updated {col_name} untuk baris {row_idx + 2}")
            else:
                print(f"⚠️ Kolom {col_name} tidak ditemukan di spreadsheet")
        except Exception as e:
            print(f"❌ Gagal mengupdate spreadsheet untuk baris {row_idx}: {str(e)}")

    # Tampilkan error summary
    if errors:
        print("\n⛔ Error Summary:")
        for error in errors:
            print(error[1])

# Jalankan proses utama
process_all_participants()
print("\n✅ Proses selesai!")